Deadline: 24 novembre ore 23:59.
Solve efficiently a generic n^2-1 puzzle using path search algorithm.

Cost=  total number of actions you need to __evaluate__. An action is something that bring me to a new state. For example the number of swaps to do.

The result is the sequence of action that took you at the end. The goal is not to find a state but a sequence of actions from srtarting point to end point: we do not look for a soluzion but for a sequence of actions.

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush

c:\Users\andrea\Documents\Computational Intelligence\CI\CI2024_lab3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [4]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 67986.44it/s]


array([[4, 7, 3],
       [1, 0, 8],
       [5, 2, 6]])

Define a function that indicates if we end the search.

In [5]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

# Depth search

Let's use a function that computes a value for each matrix so that it will be easy to store matrices.

This function shouuld be bi-directional: a number is associated to only one matrix and viceversa.

In [6]:
def matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += (10**i)*matrice[i]
    return int(score)


def simplified_matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += i*matrice[i]
    return int(score)

In [7]:
from collections import namedtuple, deque
import numpy as np

# Define action as a named tuple
action = namedtuple('Action', ['pos1', 'pos2'])

def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
    """
    Performs a depth-limited search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth limit for the search.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # # Backtrack if depth limit reached
        # if depth >= max_depth:
        #     continue
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            
            # Check if the next state has already been visited in the current path
            if matrix_score(next_state) not in visited:
                # Add the new state and path to stack, increase depth
                stack.append((next_state, path + [act], depth + 1))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(matrix_score(current_state))
    
    return None  # Return None if no solution is found within depth limit

# Iterative Deepening Depth-First Search (IDDFS) wrapper function
def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
    """
    Performs an iterative deepening DFS to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth to limit the search for IDDFS.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(initial_state, final_state, depth)
        if result is not None:
            return result
    return None


# A*


In the context of the n-puzzle problem, **Manhattan distance** is a heuristic used to estimate how close a puzzle state is to the goal. For two matrices (representing the current state and the goal state), Manhattan distance is calculated by summing up the individual distances of each tile from its target position.

1. **Tile Distance**: For each tile, the distance is the absolute difference between its current row and target row, plus the absolute difference between its current column and target column.
   
2. **Total Distance**: Summing these values for all tiles gives the Manhattan distance for the puzzle. This value represents the minimum number of moves required to reach the goal state if we could move each tile independently.

In [8]:
def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """
    Calculates the Manhattan distance for each tile in the puzzle.
    """
    distance = 0
    for value in range(1, state.size):  # Skip 0 (empty space)
        target_pos = np.argwhere(goal_state == value)[0]
        current_pos = np.argwhere(state == value)[0]
        distance += abs(target_pos[0] - current_pos[0]) + abs(target_pos[1] - current_pos[1])
    return distance

In [9]:
# Define action as a named tuple
action = namedtuple('Action', ['pos1', 'pos2'])

def a_star_search(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    """
    Performs an A* search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state, final_state), 0, initial_state.tobytes(), []))
    #this function push into open_set a new element continuing to garantee the heap properties.
    # the element that we insert are 4.
    
    visited = set()  
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        
        # Check if we reached the goal
        if np.array_equal(current_state, final_state):
            return path
        
        # Mark current state as visited
        visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            if next_bytes in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + manhattan_distance(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found

# Updated Depth search

At each iteration, among all the possible we choose the one with the minimum manahattan distance to the test goal.
In case there is no state with less manhattan distance compared to the current we select the oen that is the "best" among the possibilities. In case they are the same we use the last one.

In [10]:
def search_with_A(initial_state: np.ndarray, final_state: np.ndarray) -> list or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and calculate Manhattan distances for all possible moves
        successors = []
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = matrix_score(next_state)
            
            # Check if the next state has already been visited in the current path
            if next_score not in visited:
                # Calculate the Manhattan distance from the goal
                manhattan_dist = manhattan_distance(next_state, final_state)
                # Append (next_state, path + [act], depth + 1, manhattan_dist) to successors list
                successors.append((next_state, path + [act], depth + 1, manhattan_dist))
        
        # Sort successors by Manhattan distance (ascending)
        successors.sort(key=lambda x: x[3])  # Sort by the fourth item, which is manhattan_dist
        
        # Add sorted successors to the stack
        for next_state, new_path, new_depth, _ in successors:
            stack.append((next_state, new_path, new_depth))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(current_score)  # Uncomment if backtracking is implemented
    
    return None  # Return None if no solution is found within depth limit


Breath first ti fa trovare una soluzione ottima. Depth la prima soluzione ottima.

In [13]:
test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))
iterative_deepening_dfs(state, test_goal)

KeyboardInterrupt: 

In [12]:
a_star_search(state, test_goal)

[Action(pos1=(1, 1), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(0, 0)),
 Action(pos1=(0, 0), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(0, 2)),
 Action(pos1=(0, 2), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 0)),
 Action(pos1=(2, 0), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(0, 2)),
 Action(pos1=(0, 2), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 2))]